In [101]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict, Counter
from time import perf_counter
from scipy.sparse import csr_matrix

A synthetic corpus is required because it allows precise control over term overlap between documents, which is difficult to achieve with real-world data. This control is essential for isolating and evaluating the conditions under which an inverted index improves kNN efficiency.

In [102]:
#1) # Synthetic corpus generator
# - Used to explicitly control the degree of token overlap between documents

In [103]:
def make_synth_corpus(
    n_topics=5,         # number of topics to generate
    docs_per_topic=200, # number of documents per topic
    doc_len=80,         # number of tokens per document
    v_common=200,       # size of the shared (common) vocabulary
    v_topic=200,        # size of the topic-specific vocabulary
    p_common=0.1,       # proportion of common tokens in each document (0–1);
                        # e.g., 0.0 = no overlap, 1.0 = all tokens shared
    seed=0,             # random seed for reproducibility
):

    # Vocabulary structure

    # This section defines the types of tokens before generating any documents.
    #
    # 1) Common tokens (c*): create overlap across documents
    # 2) Topic-specific tokens (t{topic}_*): appear only within one topic

    # Step 1. Initialize RNG for reproducibility
    random_state = np.random.default_rng(seed)

    # Step 1-1. Create common vocabulary: c0, c1, c2, ...
    common_vocab = [f"c{i}" for i in range(v_common)]

    # Step 1-2. Create topic-specific vocabularies: t0_0, t0_1, t1_0, ...
    topic_vocab = {}
    for t in range(n_topics):
        topic_vocab[t] = [f"t{t}_{i}" for i in range(v_topic)]

    texts = []   # stores document texts
    labels = []  # stores topic labels


    # Document generation loop

    for topic in range(n_topics):
        for _ in range(docs_per_topic):

            # Step 2-1. Determine token counts per document
            n_common = int(doc_len * p_common)  # number of common tokens
            n_topic = doc_len - n_common        # number of topic-specific tokens

            # Step 2-2. Initialize token list
            words = []

            # Step 2-3. Sample common tokens
            words += list(
                random_state.choice(common_vocab, size=n_common, replace=True)
            )

            # Step 2-4. Sample topic-specific tokens (no cross-topic overlap)
            words += list(
                random_state.choice(topic_vocab[topic], size=n_topic, replace=True)
            )

            # Step 2-5. Shuffle token order
            random_state.shuffle(words)

            # Step 2-6. Store document as a single string
            texts.append(" ".join(words))

            # Step 2-7. Store topic label
            labels.append(topic)

    # Return generated documents and labels
    return texts, np.array(labels)


In [104]:
#Debugging
texts, labels = make_synth_corpus(n_topics=2, docs_per_topic=2, doc_len=20, p_common=0.2, seed=42)
for i in range(2):
    print(i, labels[i], texts[i])

0 0 t0_152 t0_90 t0_143 t0_171 t0_86 c17 t0_40 t0_139 t0_147 c130 t0_167 t0_105 c87 t0_25 c154 t0_102 t0_17 t0_157 t0_195 t0_18
1 0 t0_89 t0_184 t0_99 t0_148 t0_148 t0_30 t0_155 t0_109 t0_8 t0_151 t0_135 t0_72 c140 t0_178 c70 t0_136 t0_38 c13 c194 t0_93


***Pipeline overview***

1. tokenize() Splits raw text into clean tokens.
2. vectorize() Transforms tokenized documents into vector representations
3. L2 normalization() Normalizes vectors so that document length does not affect similarity.
4. Vectorize call (train/test setup) Applies the vectorization pipeline to training and test sets for temporary evaluation and experimentation.    

In [105]:
#Stopwords are removed to reduce vocabulary size and noise.
STOP = {
    "a", "an", "the",
    "and", "or", "but",
    "is", "are", "was", "were",
    "of", "to", "in", "on", "for", "with",
    "as", "by", "at", "from"
}

**Step 1. tokenize():**
Splits raw text into clean tokens and removes stopwords
to reduce noise and stabilize vectorization.

In [106]:
def tokenize(text: str, use_stopwords: bool = True, min_len: int = 2):
    text = text.lower()
    tokens = re.findall(r"[a-z0-9_]+", text.lower())

    if use_stopwords:
        tokens = [t for t in tokens if t not in STOP]

    tokens = [t for t in tokens if len(t) >= min_len]

    return tokens if tokens else ["_empty_"]

**Step 2 vectorize():**
transforms raw text into normalized TF-IDF vectors for similarity-based retrieval.

In [107]:
def vectorize(train_texts, test_texts, max_features=None):
    # Step 2-1. Configure TF-IDF vectorizer
    vec = TfidfVectorizer(
        tokenizer=tokenize,      # use custom tokenizer
        token_pattern=None,      # required when a custom tokenizer is provided
        lowercase=False,         # lowercasing is already handled in tokenize()
        max_features=max_features
    )
    
    # Step 2-2. Learn vocabulary from training data
    # Train–test splitting is intentionally kept outside this function
    # to keep the vectorization step modular and reusable.
    X_train = vec.fit_transform(train_texts)
    X_test = vec.transform(test_texts)

    # Step 2-3. L2-normalize vectors to make cosine similarity length-invariant
    X_train = normalize(X_train, norm="l2")
    X_test = normalize(X_test, norm="l2")

    # Step 2-4. Convert TF-IDF matrices to CSR format
    # for efficient row-wise operations and similarity computations.
    return vec, X_train.tocsr(), X_test.tocsr()

In [108]:
# minimal debug setup
vec, X_train, X_test = vectorize(train_texts, test_texts)

q_row = X_test[0]
k = 2

top_ids, top_scores = brute_topk(X_train, q_row, k)
print("Top-k IDs:", top_ids)
print("Top-k scores:", top_scores)


Top-k IDs: [0 1]
Top-k scores: [0.67068255 0.67068255]


***Evaluation***


*BruteForce*


*Inverted Index kNN*

Brute-force kNN (k-Nearest Neighbors) is the simplest baseline implementation of kNN. For each test (query) document, it computes the similarity (or distance) between that document and every training document, then selects the top-k most similar (or closest) training documents as the “neighbors.” The predicted label is usually determined by majority vote (classification) or averaging (regression) over those k neighbors.

In other words, brute-force kNN performs no pruning or indexing: it always evaluates all training documents per query. This makes it straightforward and exact, but potentially slow when N is large. It is commonly used as a baseline to verify that optimized methods (e.g., inverted-index candidate pruning or approximate nearest neighbor search) produce the same kNN results while reducing computation.

In [109]:
# Step 1 Brute-force kNN :This function provides an exact brute-force kNN baseline by computing cosine similarity between a query and all training documents.

In [110]:
def brute_topk(X_train, q_row, k):
    """
    Compares a query document (q_row) against all training documents (X_train)
    and returns the top-k most similar documents.
    """


    # Step 1-1 Compute similarity scores

    # Cosine similarity is computed via dot product
    # Result: one similarity score per training document
    scores = cosine_similarity(X_train, q_row).flatten()

 
    # Step 1-2 Sort document indices by score (descending)
    # argsort returns indices of sorted values
    # We use -scores to sort from highest to lowest
    sorted_doc_ids = np.argsort(-scores)


    # Step 1-3 Select top-k documents
    #e.g:
    #scores = [0.82, 0.10, 0.56, 0.33]
    #sorted_doc_ids = [0, 2, 3, 1]
    # k = 2
    
    top_k_doc_ids = sorted_doc_ids[:k]
    top_k_scores = scores[top_k_doc_ids]

    # Return top-k document indices and their similarity scores
    return top_k_doc_ids, top_k_scores

In [111]:
# Step 2 Vote : Given k nearest neighbor document indices (doc_ids),predict the label by majority voting.

In [112]:
def vote(labels, doc_ids):


    # Step 2-1.  Retrieve labels of selected documents
    selected_labels = labels[doc_ids]


    # Step 2-2 Count label occurrences
    count = Counter(selected_labels)


    # Step 2-3 Select the most frequent label
    # most_common(1) returns [(label, frequency)]
    predicted_label = count.most_common(1)[0][0]

    return predicted_label


In [113]:
#Debugging
# brute-force kNN 
top_ids, top_scores = brute_topk(X_train, q_row, k)
print("Top-k document IDs:")
print(top_ids)
print()

print("Top-k scores:")
print(top_scores)
print()


# Voting
pred_label = vote(labels, top_ids)
print("Predicted label:", pred_label)

Top-k document IDs:
[0 1]

Top-k scores:
[0.67068255 0.67068255]

Predicted label: 0


In [114]:
# Dummy train labels (for debugging / pipeline check)
y_train = np.arange(X_train.shape[0])

In [115]:
#Step 3.  inverted index :An inverted index first determines which documents are worth comparingby checking whether they share terms with the query.

In [116]:
from collections import defaultdict
from scipy.sparse import csr_matrix

def build_inv_index(X_train: csr_matrix):
    """
    Builds an inverted index from TF-IDF vectorized training data (X_train).

    Returns:
        inv : dict
            term_id -> list of document IDs
            A mapping from each term to the documents in which it appears.
    """

    # Step 3-1. Initialize inverted index
    # key   : term_id (column index in the TF-IDF matrix)
    # value : array of document IDs containing the term
    inv = defaultdict(np.ndarray)

    # Step 3-2. Convert CSR matrix to CSC matrix
    # CSR: efficient for row-based access (documents)
    # CSC: efficient for column-based access (terms)
    # Inverted index requires term-wise (column-wise) access
    Xc = X_train.tocsc()

    # Step 3-3. Iterate over all terms in the vocabulary
    # Xc.shape[1] = vocabulary size
    for t in range(Xc.shape[1]):

        # Extract the column corresponding to term t
        col = Xc.getcol(t)

        # Step 3-4. Store documents containing the term
        # col.nnz = number of non-zero entries (documents containing the term)
        if col.nnz:
            inv[t] = col.indices

    return inv


In [117]:
# Debug
# Create inverted index
inv = build_inv_index(X_train)

# Number of terms to display
n_show = 2

print("=== Inverted Index Sample ===")
for i, (term_id, doc_ids) in enumerate(inv.items()):
    print(f"term {term_id} → documents {doc_ids.tolist()}")
    if i + 1 == n_show:
        break

=== Inverted Index Sample ===
term 0 → documents [1]
term 1 → documents [0, 1]


In [118]:
import numpy as np
from scipy.sparse import csr_matrix

def get_candidates(inv, q_row: csr_matrix):

    # Step 1. Get term IDs that appear in the query document
    # q_row.indices contains term IDs with non-zero TF-IDF values
    # e.g., q_row.indices = [1, 5, 9]
    terms = q_row.indices

    # Step 2. If the query has no terms, return an empty array
    if len(terms) == 0:
        return np.array([], dtype=np.int32)

    # Step 3. Use a set to store candidate document IDs
    # (avoids duplicates across multiple query terms)
    # e.g., inv[1] -> [doc_ids]
    candidate_docs = set()

    # Step 4. For each term in the query, collect matching documents
    for t in terms:
        if t in inv:
            candidate_docs.update(inv[t])

    # Step 5. Convert the set of document IDs to a NumPy array
    return np.fromiter(candidate_docs, dtype=np.int32)


In [119]:
#Debugging
query_id = 0
q_row = X_test[query_id]

# candidate 
cand = candidates(inv, q_row)

print("Query document index:", query_id)
print("Number of candidates:", len(cand))
print("Candidate doc IDs (first 10 docs):", cand[:10])


Query document index: 0
Number of candidates: 2
Candidate doc IDs (first 10 docs): [0 1]


In [120]:
print("Total number of training documents:", X_train.shape[0])
print("Candidate ratio:", len(cand) / X_train.shape[0])

Total number of training documents: 2
Candidate ratio: 1.0


With brute-force kNN, the query is compared against all 150 training documents.

With inverted-index-based candidate selection, only the documents that share at least one term with the query are evaluated.
In this setting (p_common = 0.0), the query shares terms only with documents from the same topic, resulting in about 50 candidate documents.

As a result, the number of similarity computations is reduced from 150 to 50, achieving roughly a 3× reduction in computation without changing the kNN result.


#5) Inverted-index kNN (candidate pruning):
"Instead of comparing the query against every training document, we first collect only the candidate documents that share at least one term with the query, and then run kNN within that reduced set."


In [121]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

def inv_topk(X_train: csr_matrix, q_row: csr_matrix, inv, k: int):
    """
    kNN with candidate pruning using an inverted index.

    1) Collect candidate docs that share at least one term with the query
    2) Compute cosine similarity only for candidate docs
    3) Return top-k documents among the candidates

    Returns:
      - top-k document IDs (global doc_ids in X_train)
      - top-k scores
      - number of candidate documents
    """

    # 1) Collect candidate documents (term overlap with the query)
    cand = get_candidates(inv, q_row)

    # 2) If no candidates exist (no shared terms)
    if len(cand) == 0:
        return cand, np.array([], dtype=float), 0

    # 3) Compute scores only over candidate documents
    X_cand = X_train[cand]
    scores = cosine_similarity(X_cand, q_row).ravel()

    # 4) Select top-k by score (descending)
    if len(scores) <= k:
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k - 1)[:k]
        idx = idx[np.argsort(-scores[idx])]

    # 5) Return top-k docs, scores, and candidate count
    return cand[idx], scores[idx], len(cand)


In [122]:
# inv_topk Debugging
query_id = 0
k = 5

top_ids, top_scores, n_cand = inv_topk(X_train, X_test[query_id], inv, k)

print("Query id:", query_id, "| True label:", labels[query_id])
print("Candidates evaluated:", n_cand, "/", X_train.shape[0])
print("Top-3 (doc_id, score):", list(zip(top_ids[:3], top_scores[:3])))

Query id: 0 | True label: 0
Candidates evaluated: 2 / 2
Top-3 (doc_id, score): [(0, 0.6706825478673185), (1, 0.6706825478673185)]


In [123]:
#6) Comparison

In [124]:
inv = build_inv_index(X_train)

qi = 0
q = X_test[qi]
k = 5

b_ids, _ = brute_topk(X_train, q, k)
b_pred = vote(y_train, b_ids)

i_ids, _, csz = inv_topk(X_train, q, inv, k)
i_pred = vote(y_train, i_ids) if len(i_ids) else -1

print("Candidates:", csz)
print("Brute pred:", b_pred)
print("Inv   pred:", i_pred)


Candidates: 2
Brute pred: 0
Inv   pred: 0


***Demonstration***

Out of all training documents, only 2 were selected as candidates for comparison, indicating successful candidate pruning.

The brute-force prediction and the inverted-index prediction are identical.
This means that the full comparison (brute force) and the candidate-based comparison (inverted index) produce the same result.

In other words, accuracy is preserved while computational cost is reduced.

In [93]:
results = run_compare(X_train, labels, X_test, labels, k=5, n_queries=50, seed=42)
for k, v in results.items():
    print(f"{k}: {v}")


inv_build_time: 0.0005273330025374889
avg_brute_time: 0.0010814580018632114
avg_inv_time: 0.00043562499922700226
avg_cand_size: 2.0
median_cand_size: 2.0
same_topk_ratio: 1.0
same_pred_ratio: 1.0
n_queries: 1


In [94]:
#7) overlap ratio 조절 스윕

**Purpose**

This experiment analyzes how the degree of token overlap (p_common) affects candidate pruning efficiency and kNN performance.

Procedure

Generate synthetic corpora while varying p_common to control vocabulary overlap between documents.

Vectorize documents using TF-IDF with L2 normalization.

Compare brute-force kNN and inverted-index kNN across multiple queries.

Measure:

average candidate set size

candidate ratio (candidates / total documents)

runtime of brute vs inverted kNN

consistency of top-k results

consistency of predicted labels

In [126]:
# Step 7-0 Sweep different overlap ratios (p_common)
p_list = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2]
sweep_results = []

for p in p_list:
   
    # Step 7-1 Generate a synthetic corpus with controlled overlap
    #    p_common controls the proportion of shared tokens
    texts, labels = make_synth_corpus(
        n_topics=3,
        docs_per_topic=50,
        doc_len=40,
        p_common=p,
        seed=42
    )


    # Step 7-2  Define train / test sets
    #    - No explicit split here (controlled synthetic setup)
    #    - Use the first 50 documents as queries
    train_texts = texts
    test_texts = texts[:50]

    # Step 7-3 Vectorize documents with TF-IDF + L2 normalization
    vec, X_train, X_test = vectorize(train_texts, test_texts)


    # Step 7-4 Run brute-force vs inverted-index kNN comparison
    result = run_compare(
        X_train=X_train,
        y_train=labels,
        X_test=X_test,
        y_test=labels,
        k=5,
        n_queries=50,
        seed=0
    )


    # Step 7-5 Record overlap ratio and normalized candidate size
    result["p_common"] = p
    result["candidate_ratio"] = result["avg_cand_size"] / X_train.shape[0]

    sweep_results.append(result)

# Step 7-6 Print a concise summary of results for each p_common
print("p_common | cand_ratio | avg_cand | brute_time | inv_time | same_topk | same_pred")
for r in sweep_results:
    print(
        f"{r['p_common']:<7}  | "
        f"{r['candidate_ratio']:.3f}      | "
        f"{r['avg_cand_size']:.1f}      | "
        f"{r['avg_brute_time']:.6f}s  | "
        f"{r['avg_inv_time']:.6f}s  | "
        f"{r['same_topk_ratio']:.2f}        | "
        f"{r['same_pred_ratio']:.2f}"
    )


p_common | cand_ratio | avg_cand | brute_time | inv_time | same_topk | same_pred
0.0      | 0.333      | 50.0      | 0.000316s  | 0.000444s  | 1.00        | 1.00
0.01     | 0.333      | 50.0      | 0.000275s  | 0.000386s  | 1.00        | 1.00
0.02     | 0.333      | 50.0      | 0.000250s  | 0.000350s  | 1.00        | 1.00
0.05     | 0.347      | 52.0      | 0.000247s  | 0.000346s  | 1.00        | 1.00
0.1      | 0.384      | 57.6      | 0.000246s  | 0.000344s  | 1.00        | 1.00
0.2      | 0.514      | 77.0      | 0.000246s  | 0.000347s  | 1.00        | 1.00


**My Final Thoughts**

As the overlap ratio increases, the average candidate set size grows, reducing the computational advantage of inverted-index–based kNN.
When overlap is low, candidate pruning significantly reduces the number of similarity computations while preserving identical top-k neighbors and predictions compared to brute-force kNN.
At higher overlap ratios, most documents become candidates, and the benefit of pruning gradually diminishes.

This experiment explicitly tests the following hypothesis:

As vocabulary overlap increases (p_common ↑), candidate pruning becomes less effective, while prediction consistency remains high.

More concretely:

Low p_common

Very small candidate sets

Large speedup over brute force

Top-k neighbors and predictions are usually identical

High p_common

Candidate set size approaches the full corpus

Computational speed advantage decreases

Prediction accuracy remains stable

Overall, these results confirm the expected and desired behavior of an inverted-index–based kNN system:
efficient computation under low overlap without sacrificing correctness, and graceful degradation as overlap increases.